# A Better Dashboard for Covid-19

We've all seen the Covid-19 dashboard of major news outlets and research institutes.
Most of them also contain data and graphs about the daily growth rate.
Those, however, are misleading for untrained readers and should
either be removed or be more extensively annotated to prevent erroneous conclusions.

First let's have a look at the dashboards of major news outlets
around the world to see some examples:

<!-- 
Good job by the New York Times. Their first chart is new cases per day (https://www.nytimes.com/interactive/2020/world/coronavirus-maps.html) 
-->

## Spiegel

From <https://www.spiegel.de/thema/coronavirus/>

![Spiegel - Overview Table](spiegel.png)

As you can see it prominently features the daily growth (gray numbers)
as well as the doubling time (orange numbers), 
in an attempt to make daily growth more tangible.


## New York Times

From 
<https://www.nytimes.com/interactive/2020/04/03/upshot/coronavirus-metro-area-tracker.html>

![New York Times - Daily Growth Rate](nytimes.png)

Below the graphs it goes on:

<!-- 
> What to look for:
> Root for these to go to zero. 
> Low-seeming numbers still mean a lot of growth: 
> A 20 percent daily growth rate means 
> cases or deaths will double in less than four days.

and 
-->

> With epidemics, these rates are often more important 
> than the current totals 
> because they tell us whether things are getting better or worse. 
> A reading of 40 percent on the charts above means that, 
> on average, the number of deaths in a place 
> has been increasing by 40 percent each day. 
> A reading of 100 percent would mean that cases were doubling daily.


## The Problem of Growth Rates

What I find troubling about this is, that they write 

>these rates tell us whether things are getting better or worse

Because, while it's true that a smaller growth rate is better 
than a larger one, **any growth rate still means we're doomed**.
Therefore the graph rather conveys 
if things are getting worse slower of faster,
which is a quite morbid interpretation 
of "things getting better".
Also, it is most certainly not how most people understand it.

The root problem of the Covid-19 crisis is
that our health-care system has a limit
for people that can be treated per day.
So a more common notion would be:

Things are …     | Number of new infections are …
:---------------:|:------
… not OK         | … **above** the limit
… getting worse  | … **higher** than the day before
… getting better | … **lower** than the day before
… OK             | … **the same or lower** than the day before and below the limit

So let's check out a few graphs
of daily growth rates and their corresponding number of cases per day
as seen in the New York Times
and try to tell if things are getting better or worse
according to our definition.

In [1]:
print "Test"

"Test"

In [2]:
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Backend.Cairo

-- From utility-ht
mapAdjacent :: (a -> a -> b) -> [a] -> [b]
mapAdjacent f xs =
    zipWith f xs (tail xs)

initialNumberOfCases = 100
days = [0..15]
xLabels = map show days

: 

In [3]:
getGrowthRate1 :: [Integer] -> [(Integer, [Double])]
getGrowthRate1 = 
    fmap (\x -> (x, [100 / fromIntegral (x + 2)]))

toRenderable $ do
  layout_title .= "Daily Growth Rate 1"
  layout_x_axis . laxis_generate .= autoIndexAxis xLabels
  plot $ plotBars <$> bars [""] (getGrowthRate1 days)

: 

In [4]:
growthSeries1 :: Double -> [Integer] -> [(Integer, Double)]
growthSeries1 initialNumCases = 
    fmap (\x -> (x, if x == 0
                    then initialNumCases
                    else 1.0 + (1 / fromIntegral (x + 1))))
                    
casesPerDayCumul = scanl1 (\a b -> (fst b, snd a * snd b)) $ growthSeries1 initialNumberOfCases days
casesPerDay = mapAdjacent (\tupA tupB -> (fst tupA, snd tupB - snd tupA)) casesPerDayCumul
toRenderable $ do
  layout_title .= "Daily Number of Cases 1"
  layout_x_axis . laxis_generate .= autoIndexAxis xLabels
  plot $ plotBars <$> bars [""] (casesPerDay <&> \(f,s) -> (f,[s]))

Line 7: Use second
Found:
(fst b, snd a * snd b)
Why not:
Data.Bifunctor.second ((*) (snd a)) bLine 8: Use second
Found:
(fst tupA, snd tupB - snd tupA)
Why not:
Data.Bifunctor.second ((-) (snd tupB)) tupA

: 

In [5]:
scaleFactor = 0.8

getGrowthRate2 :: [Integer] -> [(Integer, [Double])]
getGrowthRate2 = 
    fmap (\x -> (x, [100 / ((fromIntegral x ** scaleFactor) + 2)]))
    
toRenderable $ do
  layout_title .= "Daily Growth Rate 2"
  layout_x_axis . laxis_generate .= autoIndexAxis xLabels
  plot $ plotBars <$> bars [""] (getGrowthRate2 days)

: 

In [6]:
growthSeries2 :: Double -> [Integer] -> [(Integer, Double)]
growthSeries2 initialNumCases = 
    fmap (\x -> (x, if x == 0
                    then initialNumCases
                    else 1.0 + (1 / ((fromIntegral x ** scaleFactor) + 1))))

casesPerDayCumul = scanl1 (\a b -> (fst b, snd a * snd b)) $ growthSeries2 initialNumberOfCases [0..15]
casesPerDay = mapAdjacent (\tupA tupB -> (fst tupA, snd tupB - snd tupA)) casesPerDayCumul
toRenderable $ do
  layout_title .= "Daily Number of Cases 2"
  layout_x_axis . laxis_generate .= autoIndexAxis xLabels
  plot $ plotBars <$> bars [""] (casesPerDay <&> \(f,s) -> (f,[s]))

Line 7: Use second
Found:
(fst b, snd a * snd b)
Why not:
Data.Bifunctor.second ((*) (snd a)) bLine 8: Use second
Found:
(fst tupA, snd tupB - snd tupA)
Why not:
Data.Bifunctor.second ((-) (snd tupB)) tupA

: 

In [7]:
scaleFactor = 1.1

getGrowthRate3 :: [Integer] -> [(Integer, [Double])]
getGrowthRate3 = 
    fmap (\x -> (x, [100 / (fromInteger x * scaleFactor + 2)]))
    
toRenderable $ do
  layout_title .= "Daily Growth Rate 3"
  layout_x_axis . laxis_generate .= autoIndexAxis xLabels
  plot $ plotBars <$> bars [""] (getGrowthRate3 days)

: 

In [8]:
growthSeries3 :: Double -> [Integer] -> [(Integer, Double)]
growthSeries3 initialNumCases = 
    fmap (\x -> (x, if x == 0
                    then initialNumCases
                    else 1.0 + (1 / (fromInteger x * scaleFactor + 1))))

casesPerDayCumul = scanl1 (\a b -> (fst b, snd a * snd b)) $ growthSeries3 initialNumberOfCases [0..15]
casesPerDay = mapAdjacent (\tupA tupB -> (fst tupA, snd tupB - snd tupA)) casesPerDayCumul
toRenderable $ do
  layout_title .= "Daily Number of Cases 3"
  layout_x_axis . laxis_generate .= autoIndexAxis xLabels
  plot $ plotBars <$> bars [""] (casesPerDay <&> \(f,s) -> (f,[s]))

Line 7: Use second
Found:
(fst b, snd a * snd b)
Why not:
Data.Bifunctor.second ((*) (snd a)) bLine 8: Use second
Found:
(fst tupA, snd tupB - snd tupA)
Why not:
Data.Bifunctor.second ((-) (snd tupB)) tupA

: 

# My Proposal

1. **8000 new infections** yesterday
    (At an average of 6000 / day over last 5 days)
1. Capacity limit of healthcare system: **10&nbsp;000 new infections / day**

    Based on:
    1. 50&nbsp;000 normal beds + 10&nbsp;000 ICU beds
    1. 100&nbsp;000 nurses
    1. 20&nbsp;000 doctors
    1. 30&nbsp;000 ventilators
    1. 3 weeks of hospitalization
    1. 

1. **10 days** until this limit is reached
    
    Based on average growth rate of last 5 days


**
Summed up: Percentages are mostly misleading since
our problems are caused by static limits
**